# Google Colab (GC) - Acesso

## Objetivo

Acessar a sessão da Máquina Virtual (VM) do GC usando o JupyterLab (JL), que possui mais recursos do que a interface padrão do GC.

## Introdução

* Serviço pago, com um plano limitado free. Principais atrativos:
    * [GPU](https://cloud.google.com/compute/docs/gpus#other_available_nvidia_gpu_models): usado para acelerar aplicações, geralmente Nvidia Tesla K80 ou T40
    * [TPU](https://cloud.google.com/tpu/docs/system-architecture-tpu-vm): é um acelerador usado por exemplo em *machine learning*
    * Máquina Virtual (VM) Linux, 1 *core* / 2 *threads* Intel Xeon AVX2, 12 GB RAM
    * Acesso de qualquer lugar via navegador, ex.: celular
    * O GC por padrão usa uma versão customizada limitada do Jupyter Notebook (JN), porém é possível baixar e instalar o JL padrão e configurar um proxy reverso para acessar na máquina local usando um cliente JL, através de um servidor JL na VM
* Alguns limites da versão free (os limites podem variar):
    * 12 h rodando com o navegador aberto e com atividade
    * 3 h rodando com o navegador fechado
    * 30 min de inatividade no navegador
    * Possui limite de uso da GPU, porém a Google não publica qual é esse limite, pode ser possível usar em um dia, e no dia seguinte não
    * A VM é temporária, <u>quando encerra apaga tudo</u>, é necessário manter os arquivos importantes ou no Google Drive ou na máquina local
    * A VM quando inicia ("Conecta"), é uma VM nova, padrão, "zerada"

Apesar de funcionar relativamente bem apesar dos limites, a melhor opção sempre será um computador pessoal com placa gráfica Nvidia.

## Conectando

* primeiro colocar os arquivos de exemplo em um diretório no Google Drive (GD)
* este Notebook (GC-acesso.ipynb) deve ser aberto pelo GC (e não na máquina local), usando a interface padrão
* se este arquivo for criado diretamente no GC, o GD monta automático, caso contrário o GD precisa ser montado manualmente
* esses passos precisam ser repetidos toda vez que sobe (*session connect*) uma VM nova
* o primeiro passo é entrar no GC usando o GD, configurar a GPU/CPU, conferir as sessões, e encerrrar sessões inativas
* manter tudo no GD é importante pois se a VM encerrar por qualquer motivo, os arquivos que estão nela são apagados e perde tudo
* a célula a seguir, baixa e instala o que vamos usar, e ativa uma instância do servidor JL. O htop e o mpi4py são opcionais, e serão usados nos exemplos. Outras ferramentas podem ser incluídas caso necessário

In [1]:
! pip install jupyterlab fortran-magic mpi4py pyngrok -q

     |████████████████████████████████| 8.8 MB 9.3 MB/s 
     |████████████████████████████████| 2.5 MB 37.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 745 kB 64.0 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 344 kB 71.5 MB/s 
     |████████████████████████████████| 9.7 MB 54.8 MB/s 
     |████████████████████████████████| 423 kB 73.2 MB/s 
     |████████████████████████████████| 80 kB 10.4 MB/s 
     |████████████████████████████████| 59 kB 7.1 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 271 kB 67.3 MB/s 
     |████████████████████████████████| 133 kB 69.2 MB/s 
     |████████████████████████████████| 71 kB 225 kB/s 
     |████████████████████████████████| 107 kB 74.3 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |██████████

In [2]:
! nohup jupyter-lab --allow-root --no-browser --NotebookApp.token="" --ip=0.0.0.0 &

nohup: appending output to 'nohup.out'


* A próxima célula configura o proxy reverso para permitir que o JL fique acessível em uma nova aba no navegador web, bastando clicar no link "NgrokTunnel" que aparece ao rodar a célula abaixo. É necessário ter uma conta em `ngrok.io` para obter o *token* de autenticação e inseri-lo no lugar indicado:

In [3]:
from pyngrok import ngrok
ngrok.set_auth_token("1yeDcItTVmhBKwG6eINkvqZmKVV_2XUCiLCqSQpa1vWgRbJ7K")
ngc = ngrok.connect(8888)
print(ngc)

NgrokTunnel: "http://6bbe-35-233-236-157.ngrok.io" -> "http://localhost:8888"


Desconecta o túnel:

In [4]:
tunnels = ngrok.get_tunnels()
ngrok.disconnect(tunnels[0])
ngrok.disconnect(tunnels[1])
ngrok.kill()

---

>  Notas:
> * além desta, existem outras formas de configurar um proxy reverso
> * neste ponto já é possível acessar o JupyterLab rodando no Google Colab

## Adicionalmente:

* outra opção para montar o google drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* opções para tentar evitar que a sessão encerre por inatividade no navegador:

    https://stackoverflow.com/questions/57113226/how-to-prevent-google-colab-from-disconnecting
    
    basicamente é rodar, por exemplo, o código abaixo no console do navegador (Ctrl+Shift+I) que simula que estamos clicando na interface do GC. Mesmo assim será necessário de tempos em tempos (ex.: de hora em hora) fazer alguma atividade na página, para evitar desconexão por timeout. Isso é útil, por exemplo, quando estamos esperando algum cálculo demorado terminar:

```
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

* como montar automático o GD na VM: 

    https://stackoverflow.com/questions/52808143/colab-automatic-authentication-of-connection-to-google-drive-persistent-per-n

* monta o sistema de arquivos remoto do GD na máquina local ([rclone](http://rclone.org/drive/)):

```
$ rclone mount <REMOTE_NAME>: /mnt/<REMOTE_NAME> &
```

# Sair

* <u>não esquecer de salvar no GD todos os arquivos</u>
* "Shut Down" o JL usando o menu
* no menu do GC, "Manage sessions", e "Terminate" todas as sessões